In [108]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import os 
from tqdm import tqdm
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score
#pd.set_option('display.max_columns', None)

Dans le cadre du data challenge QRT de cette année, nous vous proposons un challenge de prédiction de résultat de match. Vous recevrez des données historiques réelles au niveau des équipes et des joueurs, et vous devrez prédire quelle équipe gagne ou s'il y a un match nul.

Le score cible est l’accuracy de la prédiction pour les trois classes [HOME_WINS, DRAW, AWAY_WINS], Il existe donc pour un match trois outputs possibles, [1,0,0]. [0,1,0] et [0,0,1].

In [114]:
lien = '/Users/augustincablant/Desktop/QRT/train'
liste_df = []
for dirname, _, filenames in os.walk(lien):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        liste_df.append(pd.read_csv(lien + '/' + filename))

player_away, player_home, team_away, team_home, y_train = liste_df[0], liste_df[1], liste_df[2], liste_df[4], liste_df[3]

for col in player_away.columns:
    if col not in player_home.columns:
        print(col)

for col in team_away.columns:
    if col not in team_home.columns:
        print(col)

#Ces deux datasets ont les mêmes colonnes. On créer donc la colonne home qui vaut True ou False et on agrège les deux datasets !

player_away['HOME'], player_home['HOME'] = False, True
df_player = pd.concat([player_home, player_away], ignore_index=True).sample(frac=1).reset_index(drop=True)

team_away['HOME'], team_home['HOME'] = False, True
df_team = pd.concat([team_away, team_home], ignore_index=True).sample(frac=1).reset_index(drop=True)


##test 

lien = '/Users/augustincablant/Desktop/QRT/test'
liste_df = []
for dirname, _, filenames in os.walk(lien):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        liste_df.append(pd.read_csv(lien + '/' + filename))

team_away, player_home, team_home, player_away = liste_df[0], liste_df[1], liste_df[2], liste_df[3]

player_away['HOME'], player_home['HOME'] = False, True
df_player_test = pd.concat([player_home, player_away], ignore_index=True).sample(frac=1).reset_index(drop=True)

team_away['HOME'], team_home['HOME'] = False, True
df_team_test = pd.concat([team_away, team_home], ignore_index=True).sample(frac=1).reset_index(drop=True)

/Users/augustincablant/Desktop/QRT/train/train_away_player_statistics_df.csv
/Users/augustincablant/Desktop/QRT/train/train_home_player_statistics_df.csv
/Users/augustincablant/Desktop/QRT/train/train_away_team_statistics_df.csv
/Users/augustincablant/Desktop/QRT/train/Y_train.csv
/Users/augustincablant/Desktop/QRT/train/train_home_team_statistics_df.csv
/Users/augustincablant/Desktop/QRT/test/test_away_team_statistics_df.csv
/Users/augustincablant/Desktop/QRT/test/test_home_player_statistics_df.csv
/Users/augustincablant/Desktop/QRT/test/test_home_team_statistics_df.csv
/Users/augustincablant/Desktop/QRT/test/test_away_player_statistics_df.csv


## Idée numéro 1 

On construit un dataset sur le dataset df_team. 
Pour évalue le niveau de chaque joueur de chaque team grâce à une métrique que l'on construit, puis on fait une moyenne sur l'ensemble des joueurs de l'équipe.

Attention, dans le dataset test nous n'avons pas accès au nom des équipes. 

In [115]:
# Le but de cette cellule de code est de : 
# Construire une métrique basée sur les joueurs du match (attention bien prendre en compte away et home)
# Ajouter cette valeur dans le dataset df_team
# Faire de même pour le dataset test

#TRAIN
IDS = df_team['ID'].unique()
df_team_metric = df_team.copy()
df_team_metric['PLAYER_METRIC'] = np.nan

for id in tqdm(IDS):
    sub_true, sub_false = df_player[(df_player['ID']==id) & (df_player['HOME']==True)], df_player[(df_player['ID']==id) & (df_player['HOME']==False)]
    sub_numeric_true,  sub_numeric_false = sub_true.drop(columns=['LEAGUE', 'TEAM_NAME', 'POSITION', 'ID', 'PLAYER_NAME', 'HOME']), sub_false.drop(columns=['LEAGUE', 'TEAM_NAME', 'POSITION', 'ID', 'PLAYER_NAME', 'HOME'])
    mean_true, mean_false = sub_numeric_true.mean(axis=1, skipna=True).mean(), sub_numeric_false.mean(axis=1, skipna=True).mean()
    df_team_metric.loc[(df_team_metric['ID'] == id) & (df_team_metric['HOME'] == True), 'PLAYER_METRIC'] = mean_true
    df_team_metric.loc[(df_team_metric['ID'] == id) & (df_team_metric['HOME'] == False), 'PLAYER_METRIC'] = mean_false

#TEST (très long)
""" 
IDS_test = df_team_test['ID'].unique()
df_team_metric_test = df_team_test.copy()
df_team_metric_test['PLAYER_METRIC'] = np.nan

for id in tqdm(IDS_test):
    sub_true, sub_false = df_player_test[(df_player_test['ID']==id) & (df_player_test['HOME']==True)], df_player_test[(df_player_test['ID']==id) & (df_player_test['HOME']==False)]
    sub_numeric_true,  sub_numeric_false = sub_true.drop(columns=['POSITION', 'ID', 'HOME']), sub_false.drop(columns=['POSITION', 'ID', 'HOME'])
    mean_true, mean_false = sub_numeric_true.mean(axis=1, skipna=True).mean(), sub_numeric_false.mean(axis=1, skipna=True).mean()
    df_team_metric_test.loc[(df_team_metric_test['ID'] == id) & (df_team_metric_test['HOME'] == True), 'PLAYER_METRIC'] = mean_true
    df_team_metric_test.loc[(df_team_metric_test['ID'] == id) & (df_team_metric_test['HOME'] == False), 'PLAYER_METRIC'] = mean_false
"""

100%|██████████| 12303/12303 [00:45<00:00, 268.69it/s]


" \nIDS_test = df_team_test['ID'].unique()\ndf_team_metric_test = df_team_test.copy()\ndf_team_metric_test['PLAYER_METRIC'] = np.nan\n\nfor id in tqdm(IDS_test):\n    sub_true, sub_false = df_player_test[(df_player_test['ID']==id) & (df_player_test['HOME']==True)], df_player_test[(df_player_test['ID']==id) & (df_player_test['HOME']==False)]\n    sub_numeric_true,  sub_numeric_false = sub_true.drop(columns=['POSITION', 'ID', 'HOME']), sub_false.drop(columns=['POSITION', 'ID', 'HOME'])\n    mean_true, mean_false = sub_numeric_true.mean(axis=1, skipna=True).mean(), sub_numeric_false.mean(axis=1, skipna=True).mean()\n    df_team_metric_test.loc[(df_team_metric_test['ID'] == id) & (df_team_metric_test['HOME'] == True), 'PLAYER_METRIC'] = mean_true\n    df_team_metric_test.loc[(df_team_metric_test['ID'] == id) & (df_team_metric_test['HOME'] == False), 'PLAYER_METRIC'] = mean_false\n"

In [117]:
df_train = pd.merge(df_team_metric, y_train, on = 'ID', how = 'left').fillna(0)
df_train['TARGET'] = df_train['HOME_WINS'] - df_train['AWAY_WINS'] 
df_train['HOME'] = df_train['HOME'].astype(float)
X, y = df_train.drop(columns=['LEAGUE', 'TEAM_NAME', 'ID', 'HOME_WINS', 'DRAW', 'AWAY_WINS', 'TARGET']).copy(), df_train['TARGET']

In [118]:
rf = RandomForestClassifier()
X_train, X_test, y_tr, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

param_grid = {
    'n_estimators': [50, 100, 200, 300],
    'max_depth': [None, 10, 20, 30, 40],
    'min_samples_split': [2, 5, 10, 20],
    'min_samples_leaf': [1, 2, 4, 6, 10],
    'bootstrap': [True, False]
}
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, 
                           cv=5, n_jobs=-1, verbose=2)

grid_search.fit(X_train, y_tr)
print("Meilleurs paramètres trouvés :", grid_search.best_params_)
best_rf = grid_search.best_estimator_
y_pred = best_rf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Précision du modèle avec les meilleurs paramètres :", accuracy)

Fitting 5 folds for each of 800 candidates, totalling 4000 fits
[CV] END bootstrap=True, max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   4.5s
[CV] END bootstrap=True, max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   4.5s
[CV] END bootstrap=True, max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   4.7s
[CV] END bootstrap=True, max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   4.8s
[CV] END bootstrap=True, max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   4.8s
[CV] END bootstrap=True, max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   8.6s
[CV] END bootstrap=True, max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   8.8s
[CV] END bootstrap=True, max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total ti